<a href="https://colab.research.google.com/github/NeuralDataMind/PyTorch/blob/main/Transformations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [38]:
class TabularDataset(Dataset):
  def __init__(self, data, transform = None):
    self.data = data
    self.transform = transform

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    sample = self.data[index]
    if self.transform:
      sample = self.transform(sample)
    return sample

In [39]:
class ToTensor:
  def __call__(self, sample):
    features, label = sample[0], sample[1]
    return {'features': torch.tensor(features, dtype = torch.float32),
            'label': torch.tensor(label, dtype = torch.float32)}

In [40]:
class Normalize:
  def __call__(self, sample):
    features, label = sample[0], sample[1]
    normalized_feauters = (features - np.mean(features)) / np.std(features)
    return (normalized_feauters, label)

In [41]:
tabular_data = [(np.random.rand(2), np.random.rand()) for _ in range(100)]

In [42]:
transform = transforms.Compose([Normalize(), ToTensor()])

In [43]:
dataset = TabularDataset(data = tabular_data, transform = transform)

In [44]:
dataloader = DataLoader(dataset, batch_size = 16, shuffle = True)

In [45]:
class SimpleModel(nn.Module):
  def __init__(self, input_size):
    super(SimpleModel, self).__init__()
    self.fc = nn.Linear(input_size, 1)

  def forward(self, x):
    return self.fc(x)

In [46]:
model = SimpleModel(input_size = 2)
criterior = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [47]:
num_epochs = 50
for   epoch in range(num_epochs):
  model.train()
  total_loss = 0.0

  for batch in dataloader:
    features, labels = batch['features'], batch['label']

    optimizer.zero_grad()
    outputs = model(features)

    loss = criterior(outputs, labels.view(-1, 1))
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  average_loss = total_loss / len(dataloader)
  print(f"Epoch [{epoch + 1} / {num_epochs}], Loss: {average_loss}")

Epoch [1 / 50], Loss: 0.8350030183792114
Epoch [2 / 50], Loss: 0.6799596633229937
Epoch [3 / 50], Loss: 0.5398505415235247
Epoch [4 / 50], Loss: 0.411620774439403
Epoch [5 / 50], Loss: 0.3582664004393986
Epoch [6 / 50], Loss: 0.3028465509414673
Epoch [7 / 50], Loss: 0.23868403477328165
Epoch [8 / 50], Loss: 0.19414107714380538
Epoch [9 / 50], Loss: 0.17627188776220595
Epoch [10 / 50], Loss: 0.1458121602024351
Epoch [11 / 50], Loss: 0.12254545199019569
Epoch [12 / 50], Loss: 0.11395400549684252
Epoch [13 / 50], Loss: 0.1089726026569094
Epoch [14 / 50], Loss: 0.1001017125589507
Epoch [15 / 50], Loss: 0.08739418855735234
Epoch [16 / 50], Loss: 0.09481981503111976
Epoch [17 / 50], Loss: 0.08533501412187304
Epoch [18 / 50], Loss: 0.08098826184868813
Epoch [19 / 50], Loss: 0.08475495874881744
Epoch [20 / 50], Loss: 0.08317003079823085
Epoch [21 / 50], Loss: 0.0813336478812354
Epoch [22 / 50], Loss: 0.07858238475663322
Epoch [23 / 50], Loss: 0.08146904621805463
Epoch [24 / 50], Loss: 0.070658

In [49]:
model.eval()

with torch.no_grad():
  total_loss = 0.0

  for batch in dataloader:
    features, labels = batch['features'], batch['label']
    outputs = model(features)
    loss = criterior(outputs, labels.view(-1, 1))
    total_loss += loss.item()

  average_loss = total_loss / len(dataloader)
  print(average_loss)

0.07657798592533384
